In [11]:
from common.functions import *
class SoftmaxWithLoss:
    #インスタンス変数に取っておく
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        # forwardの式
        # -sum ( t * log (y))
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        print("backword")
        print(self.t)
        # backwardの式
        # yi - ti (iはIndex)
        batch_size = self.t.shape[0]
        print("bag")
        print(batch_size)
        # Backwardを実装して、微分値をdxに代入してください
        dx = (self.y - self.t) / batch_size
        return dx

In [12]:
import numpy as np

from dataset.mnist import load_mnist
from common.layers import *
from swl import *
from twn import *

def gradient(network, x, t):
    print("grad")
    # 自分で実装したSoftmax with lossクラスを使ってみてください
    lastLayer = SoftmaxWithLoss()
    print("after swl")
    # forward
    #self.loss(x, t)
    network.loss(x, t)
    # backward
    dout = 1
    print("beforeback")
    print(dout)
    dout = lastLayer.backward(dout)
    print("after back")
    print(dout)
    #layers = list(self.layers.values())
    layers = list(network.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
    # 設定
    grads = {}
    #grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
    grads['W1'], grads['b1'] = network.layers['Affine1'].dW, self.layers['Affine1'].db
    #grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
    grads['W2'], grads['b2'] = network.layers['Affine2'].dW, self.layers['Affine2'].db
    return grads
    

In [13]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    # 勾配
    #grad = network.numerical_gradient(x_batch, t_batch)
    #grad = gradient(x_batch, t_batch)
    print("before grad")
    grad = gradient(network, x_batch, t_batch)
    print(grad)
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

before grad
grad
after swl
beforeback
1


AttributeError: 'NoneType' object has no attribute 'shape'